In [ ]:
import time

class Sudoku1:
    def __init__(self, matrix):
        self.result = [ col for row in matrix for col in row ]
        self.history = []
        self.pending = [ 0 for row in matrix for col in row ]
        self._init_pending()
        
    def _checkbyid(self,index,num):
        row = index//9; col = index%9
        return  True if num in self.result[row*9:row*9+9] else \
                True if num in [ self.result[i*9+col] for i in range(9) ] else \
                True if num in [ self.result[row//3*27+col//3*3+9*br+bc] for br in range(3) for bc in range(3) ] else \
                False   

    def _init_pending(self):
        for i in range(81):
            if self.result[i] == 0:
                self.pending[i] = [[],True]
                for num in range(1,10):
                    if self._checkbyid(i,num)==False:
                        self.pending[i][0].append(num)
    
    def run(self):
        while True:
            for _index in range(len(self.pending)):
                node = self.pending[_index] # [ [numlist], Status Flag]
                if node != 0: # is list
                    if (len(node[0])>0)and(node[1]==True):
                        break
                node = None
            if node == None:
                return self.result
            
            numlist, s_flag = node
            num = numlist.pop()
            self.history.append([_index,num])
            
            if self._checkbyid(_index,num): # 冲突
                if len(numlist) > 0: # 尚有剩余元素，再取一个
                    continue
                index = _index
                while True: 
                    while (len(self.history)>0) and (self.history[-1][0] == index): # rollback all nums
                        self.pending[index][0].append(self.history.pop()[1])
                        
                    if len(self.history) == 0: # 回溯列表为空，说明已回溯到第一个空格，代表数独无解
                        print('数独无解')
                        return 

                    index, _ = self.history[-1] # 获得上一位置的行、列以及已压栈数据等信息
                    self.pending[index][1] = True     # 下一步继续从上一位置取数
                    self.result[index] = 0            # 清除结果表中的相应位置数据

                    if len(self.pending[index][0]) != 0: # 如果上一位置待测试数据已空，继续向上回溯
                        break
            else:
                self.pending[_index][1] = False
                self.result[_index] = num
                
                
sudoku_data = [
     [8,0,0,0,0,0,0,0,0],
     [0,0,3,6,0,0,0,0,0],
     [0,7,0,0,9,0,2,0,0],
     [0,5,0,0,0,7,0,0,0],
     [0,0,0,0,4,5,7,0,0],
     [0,0,0,1,0,0,0,3,0],
     [0,0,1,0,0,0,0,6,8],
     [0,0,8,5,0,0,0,1,0],
     [0,9,0,0,0,0,4,0,0] 
]

def reshape(matrix):
    sudoku_data = []
    for i in range(9):
        sudoku_data.append([ int(i) for i in matrix[i*9:i*9+9] ])
    return sudoku_data

sudoku = Sudoku1(sudoku_data)
for i in reshape_matrix(sudoku.run()):
    print(i)